In [1]:
### Basic Imports
import pandas as pd
import numpy as np

### Graphic libraries
import matplotlib.pyplot as plt
import seaborn as sns 

import statistics 


### Importing the relevant ML libraries 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import mean_squared_error

from sklearn.metrics import accuracy_score,roc_curve,auc,recall_score,f1_score,precision_score,classification_report,confusion_matrix,auc

import statsmodels.api as sm

### Some cosmetics add-ons
plt.style.use('fivethirtyeight')
%matplotlib inline

# Load dataset

In [2]:
df = pd.read_csv('nba_2020_clean.csv')

# Get significant features

In [27]:
features_list = ['DIS_ELO_x', 'HOME_COURT_x', 'DIS_OFFRATE_x', 'DIS_DEFRATE_x', 'DIS_PTS_x', 'DIS_AST_x', 'DIS_OREB_x', 'DIS_DREB_x']
target = 'WL_x'

train_data = df[(df['GAME_PLAYED_x'] <= 41) | (df['GAME_PLAYED_y'] <= 41 )]
test_data = df[(df['GAME_PLAYED_x'] > 41) & (df['GAME_PLAYED_y'] > 41 )]

# Creating our independent and dependent variables
x = train_data[features_list]
y = train_data['PLUS_MINUS_x']

model = sm.OLS(y,x)
results = model.fit()

features_list = []
for i in range(len(x.keys())):
    if results.pvalues[i] <= 0.1:
        features_list.append(model.exog_names[i])

features_list


['DIS_ELO_x', 'HOME_COURT_x', 'DIS_OFFRATE_x']

In [28]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:           PLUS_MINUS_x   R-squared (uncentered):                   0.086
Model:                            OLS   Adj. R-squared (uncentered):              0.073
Method:                 Least Squares   F-statistic:                              6.999
Date:                Wed, 31 Mar 2021   Prob (F-statistic):                    7.59e-09
Time:                        00:29:01   Log-Likelihood:                         -2462.4
No. Observations:                 605   AIC:                                      4941.
Df Residuals:                     597   BIC:                                      4976.
Df Model:                           8                                                  
Covariance Type:            nonrobust                                                  
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
DIS_ELO_x         0.0337      0.012      2.855      0.004       0.011       0.057
HOME_COURT_x      1.4695      0.830      1.770      0.077      -0.161       3.100
DIS_OFFRATE_x    40.0128     12.594      3.177      0.002      15.279      64.746
DIS_DEFRATE_x    -3.8933      2.549     -1.528      0.127      -8.898       1.112
DIS_PTS_x        -0.0845      0.096     -0.882      0.378      -0.273       0.104
DIS_AST_x         0.1900      0.213      0.892      0.373      -0.229       0.609
DIS_OREB_x       -0.2326      0.271     -0.859      0.391      -0.765       0.299
DIS_DREB_x        0.0383      0.220      0.174      0.862      -0.394       0.471
==============================================================================
Omnibus:                        4.919   Durbin-Watson:                   1.946
Prob(Omnibus):                  0.085   Jarque-Bera (JB):                5.287
Skew:                          -0.132   Prob(JB):                       0.0711
Kurtosis:                       3.374   Cond. No.                     1.41e+03
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The condition number is large, 1.41e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

# Get performance of each model

In [29]:
models_dict = {
    'Linear Regression': LinearRegression(),
    'Logistic Regression':LogisticRegression(),
    'Naive Bayes':GaussianNB(),
    'Decision Trees':DecisionTreeClassifier(),
    'SVM linear': svm.SVC(kernel='linear'),
    'SVM rbf': svm.SVC(kernel='rbf'),
    'Random Forest': RandomForestClassifier(n_estimators = 100),
    # 'XGBoost': xgb.XGBClassifier(use_label_encoder=False)
}

performance_table = {'f1':[],'acc':[]}

for model_name in models_dict:
    X_train = train_data[features_list]
    X_test = test_data[features_list]
    y_train = train_data['WL_x']
    y_test = test_data['WL_x']
    
    m = models_dict[model_name]

    if model_name == 'Linear Regression':
        y_train = train_data['PLUS_MINUS_x']
    
    m.fit(X_train, y_train)
    predictions = m.predict(X_test)

    if model_name == 'Linear Regression':
        for i, v in enumerate(predictions):
            if v > 0:
                predictions[i] = 1
            else:
                predictions[i] = 0
    
    f1 = round(f1_score(y_test,predictions),4)
    acc = round(accuracy_score(y_test,predictions),4)
    
    performance_table['f1'].append(f1)
    performance_table['acc'].append(acc)

# for k, v in performance_table.items():
#     performance_df = pd.DataFrame(v,index=list(models_dict.keys()))
#     print(performance_df)

performance_df = pd.DataFrame(performance_table, index=list(models_dict.keys()))
performance_df

,f1,acc
Linear Regression,0.7792,0.7424
Logistic Regression,0.7532,0.7121
Naive Bayes,0.7792,0.7424
Decision Trees,0.6494,0.5909
SVM linear,0.7467,0.7121
SVM rbf,0.7561,0.6970
Random Forest,0.6923,0.6364


# Add ensemble model prediction column for each row in test data

In [30]:
models_dict = {
    'Linear Regression': LinearRegression(),
    'Logistic Regression':LogisticRegression(),
    'Naive Bayes':GaussianNB(),
    # 'Decision Trees':DecisionTreeClassifier(),
    'SVM linear': svm.SVC(kernel='linear'),
    'SVM rbf': svm.SVC(kernel='rbf'),
    # 'Random Forest': RandomForestClassifier(n_estimators = 100),
    # 'XGBoost': xgb.XGBClassifier(use_label_encoder=False)
}
test_data['Prediction'] = 0
for index, row in test_data.iterrows():
    test_df = row.to_frame().T
    prediction_data = {}
    for model_name in models_dict:
        X_train = train_data[features_list]
        X_test = test_df[features_list]
        y_train = train_data['WL_x']
        
        m = models_dict[model_name]

        if model_name == 'Linear Regression':
            y_train = train_data['PLUS_MINUS_x']
        
        m.fit(X_train, y_train)
        prediction = m.predict(X_test)

        if model_name == 'Linear Regression':
            if prediction[0] > 0:
                prediction[0] = 1
            else:
                prediction[0] = 0
                        
        prediction_data[model_name] = prediction[0]
        
    final_prediction = 0
    for k, v in prediction_data.items():
        final_prediction += v

    final_prediction = round(final_prediction / 5)
    test_data['Prediction'][index] = final_prediction


/python/site-packages/IPython/core/interactiveshell.py:3427: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
<ipython-input-30-517efc1a02b1>:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['Prediction'][index] = final_prediction
/Users/samuelsim/Library/Python/3.8/lib/python/site-packages/IPython/core/interactiveshell.py:3427: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec

# Get performance of ensemble model

In [31]:
y_test = test_data['WL_x']
f1 = f1_score(y_test,test_data['Prediction'])
acc = accuracy_score(y_test,test_data['Prediction'])
print("f1 score: {}\nacc score: {}".format(round(f1,4),round(acc,4)))

f1 score: 0.7632
acc score: 0.7273


# Ensemble performance - prediction for every game based on all previous games

In [11]:
features_list = ['DIS_ELO_x', 'HOME_COURT_x', 'DIS_OFFRATE_x', 'DIS_DEFRATE_x', 'DIS_PTS_x', 'DIS_AST_x', 'DIS_OREB_x', 'DIS_DREB_x']
target = 'WL_x'

# predict and test 2nd half of season
test_data = df[(df['GAME_PLAYED_x'] > 41) & (df['GAME_PLAYED_y'] > 41)]
test_data['Prediction'] = 0

models_dict = {
    'Linear Regression': LinearRegression(),
    'Logistic Regression':LogisticRegression(),
    'Naive Bayes':GaussianNB(),
    # 'Decision Trees':DecisionTreeClassifier(),
    'SVM linear': svm.SVC(kernel='linear'),
    'SVM rbf': svm.SVC(kernel='rbf'),
    # 'Random Forest': RandomForestClassifier(n_estimators = 100),
    # 'XGBoost': xgb.XGBClassifier(use_label_encoder=False)
}

for index, row in test_data.iterrows():
    # Creating training dataset for all previous games
    idx = df.index[df['GAME_ID'] == row['GAME_ID']]
    train_df = df.loc[:idx[0]-1]
    test_df = row.to_frame().T

    # Creating our independent and dependent variables
    x = train_df[features_list]
    y = train_df['PLUS_MINUS_x']

    model = sm.OLS(y,x)
    results = model.fit()

    # Extracting significant features
    features_list = []
    for i in range(len(x.keys())):
        if results.pvalues[i] <= 0.1:
            features_list.append(model.exog_names[i])

    # Predicting game outcome
    prediction_data = {}
    for model_name in models_dict:
        X_train = train_df[features_list]
        y_train = train_df['WL_x']
        X_test = test_df[features_list]

        m = models_dict[model_name]

        if model_name == 'Linear Regression':
            y_train = train_df['PLUS_MINUS_x']
        
        m.fit(X_train, y_train)
        prediction = m.predict(X_test)

        if model_name == 'Linear Regression':
            if prediction[0] > 0:
                prediction[0] = 1
            else:
                prediction[0] = 0
                        
        prediction_data[model_name] = prediction[0]
        
    final_prediction = 0
    for k, v in prediction_data.items():
        final_prediction += v

    final_prediction = round(final_prediction / 5)

    # Appending predicted outcome to test_data
    test_data['Prediction'][index] = final_prediction


/python/site-packages/IPython/core/interactiveshell.py:3427: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
<ipython-input-11-f2f3dd949f9c>:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['Prediction'][index] = final_prediction
/Users/samuelsim/Library/Python/3.8/lib/python/site-packages/IPython/core/interactiveshell.py:3427: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec

In [12]:
y_test = test_data['WL_x']

f1 = f1_score(y_test,test_data['Prediction'])
accuracy = accuracy_score(y_test,test_data['Prediction'])
precision = precision_score(y_test,test_data['Prediction'])
recall = recall_score(y_test,test_data['Prediction'])

print("Ensemble model")
print("-"*20)
print("accuracy: {}\nprecision: {}\nrecall: {}\nf1 score: {}".format(round(accuracy,4),round(precision,4),round(recall,4),round(f1,4)))

Ensemble model
--------------------
accuracy: 0.6602
precision: 0.6685
recall: 0.6503
f1 score: 0.6593


In [13]:
test_data

,SEASON_ID_x,TEAM_ID_x,TEAM_ABBREVIATION_x,TEAM_NAME_x,GAME_ID,GAME_DATE_x,MATCHUP_x,WL_x,MIN_x,PTS_x,...,DEFRATE_y,ELO_y,DIS_PTS_y,DIS_AST_y,DIS_OREB_y,DIS_DREB_y,DIS_OFFRATE_y,DIS_DEFRATE_y,DIS_ELO_y,Prediction
592,22020,1610612758,SAC,Sacramento Kings,22000641,2021-03-20,SAC @ PHI,0,240,105,...,1.137078,1595.038811,0.365854,-2.121951,-0.024390,3.707317,-0.009096,0.035032,140.548588,0
596,22020,1610612743,DEN,Denver Nuggets,22000646,2021-03-21,DEN vs. NOP,0,241,108,...,1.118229,1455.242649,-0.682927,-0.658537,1.609756,0.829268,-0.012037,-0.000385,-118.329024,1
597,22020,1610612738,BOS,Boston Celtics,22000167,2021-03-21,BOS vs. ORL,1,241,112,...,1.206567,1409.834103,-6.975610,0.121951,-0.707317,3.390244,-0.073447,0.051740,-69.109142,1
599,22020,1610612739,CLE,Cleveland Cavaliers,22000648,2021-03-21,CLE vs. TOR,1,241,116,...,1.161803,1440.647729,8.682927,2.219512,-2.073171,-0.121951,0.071374,0.129877,17.220144,0
604,22020,1610612755,PHI,Philadelphia 76ers,22000650,2021-03-21,PHI @ NYK,1,263,101,...,1.026423,1499.751511,-10.619048,-2.761905,-0.071429,-0.476190,-0.044262,-0.092025,-101.449105,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
962,22020,1610612755,PHI,Philadelphia 76ers,22000994,2021-05-05,PHI @ HOU,1,241,135,...,1.033823,1320.193462,-5.861538,-0.400000,-0.784615,-1.784615,-0.069459,-0.060712,-293.836399,1
963,22020,1610612753,ORL,Orlando Magic,22000990,2021-05-05,ORL vs. BOS,0,239,96,...,1.136973,1519.743843,8.430769,1.369231,0.569231,-1.323077,0.080801,0.059902,139.270204,0
964,22020,1610612764,WAS,Washington Wizards,22000235,2021-05-05,WAS @ MIL,0,239,134,...,1.148345,1591.369894,3.584615,0.061538,0.723077,2.507692,0.056601,0.073545,76.778705,0
965,22020,1610612743,DEN,Denver Nuggets,22000995,2021-05-05,DEN vs. NYK,1,240,113,...,1.144881,1576.090413,-7.661538,-5.630769,-0.446154,1.246154,-0.055856,0.058909,-43.391419,1
